In [1]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, Dropdown
from ipywidgets import Layout

In [2]:
def generate_noise(t, mean, covariance):
    return np.random.normal(mean, np.sqrt(covariance), size=len(t))

def harmonic_signal(t, amplitude, frequency, phase):
    return amplitude * np.sin(2 * np.pi * frequency * t + phase)

def split_and_color_filter(y_data):
    y_upper = np.maximum(y_data, 0)
    y_lower = np.minimum(y_data, 0)
    return y_upper, y_lower

t = np.linspace(0, 10, 500)
noise = np.zeros_like(t)
prev_noise_params = {"mean": 0.0, "covariance": 0.1}

def update_noise(noise_mean, noise_covariance):
    global noise, prev_noise_params
    if noise_mean != prev_noise_params["mean"] or noise_covariance != prev_noise_params["covariance"]:
        noise = generate_noise(t, noise_mean, noise_covariance)
        prev_noise_params["mean"] = noise_mean
        prev_noise_params["covariance"] = noise_covariance

def update_graph(graph_type, amplitude, frequency, phase, noise_mean, noise_covariance):
    update_noise(noise_mean, noise_covariance)

    y_clean = harmonic_signal(t, amplitude, frequency, phase)
    y_noisy = y_clean + noise
    y_upper, y_lower = split_and_color_filter(y_noisy)

    fig = go.Figure()

    if graph_type == "Graph with Noise":
        fig.add_trace(go.Scatter(x=t, y=y_noisy, mode='lines', name='Graph with Noise', line=dict(color="green")))
    elif graph_type == "Clean graph":
        fig.add_trace(go.Scatter(x=t, y=y_clean, mode='lines', name='Clean graph', line=dict(color="blue")))
    elif graph_type == "Filtered graph":
        fig.add_trace(go.Scatter(x=t, y=y_upper, mode='lines', name='Upper Part', line=dict(color="green")))
        fig.add_trace(go.Scatter(x=t, y=y_lower, mode='lines', name='Lower Part', line=dict(color="#c41d1d")))

    fig.update_layout(
        title={
            'text': "<b>The New Year is coming</b><br>🎄🎅🏻❄️☃️🎄",
            'y': 0.9,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        title_font=dict(size=20, color="#003366"),
        plot_bgcolor="#f0f8ff",
        xaxis=dict(
            gridcolor="#add8e6",
            range=[0, 10],
            showticklabels=True,
            showline=False,
            title=None
        ),
        yaxis=dict(
            gridcolor="#add8e6",
            range=[-2, 2],
            showticklabels=True,
            showline=False,
            title=None
        ),
        margin=dict(l=50, r=50, t=80, b=50),
        paper_bgcolor="white",
        shapes=[
            dict(
                type="rect",
                xref="paper", yref="paper",
                x0=0, y0=0, x1=1, y1=1,
                line=dict(color="green", width=4),
                layer="above"
            )
        ]
    )
    fig.show()

interact(
    update_graph,
    graph_type=Dropdown(
        options=["Graph with Noise", "Clean graph", "Filtered graph"],
        value="Graph with Noise",
        description="Graph Type"
    ),
    amplitude=FloatSlider(
        min=0.5, max=2.0, step=0.001, value=1.0,
        layout=Layout(width="800px"), readout_format=".3f",
        style={'handle_color': 'red'}
    ),
    frequency=FloatSlider(
        min=0.1, max=2.0, step=0.001, value=1.0,
        layout=Layout(width="800px"), readout_format=".3f",
        style={'handle_color': 'green'}
    ),
    phase=FloatSlider(
        min=-np.pi, max=np.pi, step=0.001, value=0.0,
        layout=Layout(width="800px"), readout_format=".3f",
        style={'handle_color': 'red'}
    ),
    noise_mean=FloatSlider(
        min=-1.0, max=1.0, step=0.001, value=0.0,
        layout=Layout(width="800px"), readout_format=".3f",
        style={'handle_color': 'green'}
    ),
    noise_covariance=FloatSlider(
        min=0.01, max=1.0, step=0.001, value=0.1,
        layout=Layout(width="800px"), readout_format=".3f",
        style={'handle_color': 'red'}
    )
)


interactive(children=(Dropdown(description='Graph Type', options=('Graph with Noise', 'Clean graph', 'Filtered…

<function __main__.update_graph(graph_type, amplitude, frequency, phase, noise_mean, noise_covariance)>

In [3]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, Dropdown
from ipywidgets import Layout
from scipy import signal  # Імпортуємо для Notch-фільтра

# Генерація шуму
def generate_noise(t, mean, covariance):
    return np.random.normal(mean, np.sqrt(covariance), size=len(t))

# Генерація гармонічного сигналу
def harmonic_signal(t, amplitude, frequency, phase):
    return amplitude * np.sin(2 * np.pi * frequency * t + phase)

# Функція для розділення сигналу на верхню та нижню частини
def split_and_color_filter(y_data):
    y_upper = np.maximum(y_data, 0)
    y_lower = np.minimum(y_data, 0)
    return y_upper, y_lower

# Функція для застосування Notch-фільтра
def apply_notch_filter(data, notch_freq, Q, fs=250):
    w0 = notch_freq / (fs / 2)  # Нормалізована частота
    b, a = signal.iirnotch(w0, Q)  # Створення Notch-фільтра з параметром Q
    filtered_data = signal.filtfilt(b, a, data)  # Застосування фільтра
    return filtered_data

# Часова шкала
t = np.linspace(0, 10, 500)
noise = np.zeros_like(t)
prev_noise_params = {"mean": 0.0, "covariance": 0.1}

# Оновлення шуму
def update_noise(noise_mean, noise_covariance):
    global noise, prev_noise_params
    if noise_mean != prev_noise_params["mean"] or noise_covariance != prev_noise_params["covariance"]:
        noise = generate_noise(t, noise_mean, noise_covariance)
        prev_noise_params["mean"] = noise_mean
        prev_noise_params["covariance"] = noise_covariance

# Оновлення графіка
def update_graph(graph_type, amplitude, frequency, phase, noise_mean, noise_covariance, notch_freq, Q):
    update_noise(noise_mean, noise_covariance)

    y_clean = harmonic_signal(t, amplitude, frequency, phase)
    y_noisy = y_clean + noise
    y_upper, y_lower = split_and_color_filter(y_noisy)

    # Фільтрація сигналу за допомогою Notch-фільтра
    y_filtered = apply_notch_filter(y_noisy, notch_freq, Q)

    fig = go.Figure()

    if graph_type == "Graph with Noise":
        fig.add_trace(go.Scatter(x=t, y=y_noisy, mode='lines', name='Graph with Noise', line=dict(color="green")))
    elif graph_type == "Clean graph":
        fig.add_trace(go.Scatter(x=t, y=y_clean, mode='lines', name='Clean graph', line=dict(color="blue")))
    elif graph_type == "Filtered graph":
        fig.add_trace(go.Scatter(x=t, y=y_upper, mode='lines', name='Upper Part', line=dict(color="green")))
        fig.add_trace(go.Scatter(x=t, y=y_lower, mode='lines', name='Lower Part', line=dict(color="#c41d1d")))
    elif graph_type == "Notch Filtered graph":
        fig.add_trace(go.Scatter(x=t, y=y_filtered, mode='lines', name='Notch Filtered', line=dict(color="purple")))

    fig.update_layout(
        title={
            'text': "<b>The New Year is coming</b><br>🎄🎅🏻❄️☃️🎄",
            'y': 0.9,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        title_font=dict(size=20, color="#003366"),
        plot_bgcolor="#f0f8ff",
        xaxis=dict(
            gridcolor="#add8e6",
            range=[0, 10],
            showticklabels=True,
            showline=False,
            title=None
        ),
        yaxis=dict(
            gridcolor="#add8e6",
            range=[-2, 2],
            showticklabels=True,
            showline=False,
            title=None
        ),
        margin=dict(l=50, r=50, t=80, b=50),
        paper_bgcolor="white",
        shapes=[
            dict(
                type="rect",
                xref="paper", yref="paper",
                x0=0, y0=0, x1=1, y1=1,
                line=dict(color="green", width=4),
                layer="above"
            )
        ]
    )
    fig.show()

# Інтерфейс
interact(
    update_graph,
    graph_type=Dropdown(
        options=["Graph with Noise", "Clean graph", "Filtered graph", "Notch Filtered graph"],
        value="Graph with Noise",
        description="Graph Type"
    ),
    amplitude=FloatSlider(
        min=0.5, max=2.0, step=0.001, value=1.0,
        layout=Layout(width="800px"), readout_format=".3f",
        style={'handle_color': 'red'}
    ),
    frequency=FloatSlider(
        min=0.1, max=2.0, step=0.001, value=1.0,
        layout=Layout(width="800px"), readout_format=".3f",
        style={'handle_color': 'green'}
    ),
    phase=FloatSlider(
        min=-np.pi, max=np.pi, step=0.001, value=0.0,
        layout=Layout(width="800px"), readout_format=".3f",
        style={'handle_color': 'red'}
    ),
    noise_mean=FloatSlider(
        min=-1.0, max=1.0, step=0.001, value=0.0,
        layout=Layout(width="800px"), readout_format=".3f",
        style={'handle_color': 'green'}
    ),
    noise_covariance=FloatSlider(
        min=0.01, max=1.0, step=0.001, value=0.1,
        layout=Layout(width="800px"), readout_format=".3f",
        style={'handle_color': 'red'}
    ),
    notch_freq=FloatSlider(
        min=0.5, max=100.0, step=0.1, value=50.0,
        layout=Layout(width="800px"), readout_format=".3f",
        style={'handle_color': 'blue'}
    ),
    Q=FloatSlider(
        min=1.0, max=100.0, step=0.1, value=30.0,
        layout=Layout(width="800px"), readout_format=".3f",
        style={'handle_color': 'blue'}
    )
)


interactive(children=(Dropdown(description='Graph Type', options=('Graph with Noise', 'Clean graph', 'Filtered…

<function __main__.update_graph(graph_type, amplitude, frequency, phase, noise_mean, noise_covariance, notch_freq, Q)>